### 0. Understand your enviroment

In [ ]:
# Look at the current folder
pwd()

Note that the package folder is one level up from the notebooks folder and two levels from the current directory (if this file's location has not changed). The package's core code is in the `src` folder. Using Julia's package manager `Pkg` we activate the project two levels up by passing the path to the `Pkg.activate` function.

### 1. Activate the IceFloeTracker package

First activate the project environment, load things, and set up some paths.

In [ ]:
# The notebook presently uses test images and some testing tools from the test directory.
HOME = "../.." # path to the root of the project two levels up
TEST = joinpath(HOME, "test")

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

# Load packages

using IceFloeTracker
using IceFloeTracker: float64, @test_approx_eq_sigma_eps, load, Gray
using DelimitedFiles # for loading test data in csv format
using Distributed # for parallel processing

addprocs(1) # add a single worker process for watershed computation
include(joinpath(TEST,"config.jl")) # loads some test data paths
include(joinpath(TEST,"test_error_rate.jl")) # loads the `test_error_rate` function

imshow(img) = Gray.(img)

In [ ]:
# For tiling
using IceFloeTracker:
    adapthisteq_params,
    adjust_gamma_params,
    brighten_factor,
    ice_labels_thresholds,
    ice_masks_params,
    prelim_icemask_params,
    preprocess_tiling,
    structuring_elements,
    unsharp_mask_params,
    get_tiles,
    _get_masks,
    red,
    green,
    blue,
    to_uint8,
    _process_image_tiles,
    rgb2gray,
    unsharp_mask,
    reconstruct,
    get_brighten_mask,
    imbrighten,
    adjustgamma,
    imcomplement,
    get_ice_masks,
    get_segment_mask,
    watershed2,
    _regularize,
    get_final


In [ ]:
# Set up test data
region = (1016:3045, 1486:3714)
data_dir = joinpath(TEST, "test_inputs")
true_color_image = load(
    joinpath(data_dir, "NE_Greenland_truecolor.2020162.aqua.250m.tiff")
)
ref_image = load(joinpath(data_dir, "NE_Greenland_reflectance.2020162.aqua.250m.tiff"))
landmask = float64.(load(joinpath(data_dir, "matlab_landmask.png"))) .> 0

# Crop images to region of interest
true_color_image, ref_image, landmask = [
    img[region...] for img in (true_color_image, ref_image, landmask)
]

landmask = (dilated=landmask,)
tiles = get_tiles(true_color_image; rblocks=2, cblocks=3)

In [ ]:
begin
    @debug "Step 1/2: Get masks"
    mask_cloud_ice, clouds_view = _get_masks(
        float64.(ref_image); ice_labels_thresholds...
    )
    clouds_view .= .!mask_cloud_ice .* clouds_view

    # Get clouds_red for adaptive histogram equalization
    ref_img_cloudmasked = ref_image .* .!clouds_view
end

In [ ]:
begin
    @debug "Step 3: Tiled adaptive histogram equalization"
    clouds_red = to_uint8(float64.(red.(ref_img_cloudmasked) .* 255))
    clouds_red[landmask.dilated] .= 0

    rgbchannels = _process_image_tiles(
        true_color_image, clouds_red, tiles, adapthisteq_params...
    )

    gammagreen = @view rgbchannels[:, :, 2]
    equalized_gray = rgb2gray(rgbchannels)
end

imshow(equalized_gray/255)

In [ ]:
begin
    @debug "Step 4: Remove clouds from equalized_gray"
    masks = [f.(ref_img_cloudmasked) .== 0 for f in [red, green, blue]]
    combo_mask = reduce((a, b) -> a .& b, masks)
    equalized_gray[combo_mask] .= 0
end

imshow(equalized_gray/255)


In [ ]:
begin
    @debug "Step 5: unsharp_mask on equalized_gray and reconstruct"
    sharpened = to_uint8(unsharp_mask(equalized_gray, unsharp_mask_params...))
    equalized_gray_sharpened_reconstructed = reconstruct(
        sharpened, structuring_elements.se_disk1, "dilation", true
    )
    equalized_gray_sharpened_reconstructed[landmask.dilated] .= 0
end

imshow(equalized_gray_sharpened_reconstructed/255)

In [ ]:
# TODO: Steps 6 and 7 can be done in parallel as they are independent
begin
    @debug "Step 6: Repeat step 5 with equalized_gray (landmasking, no sharpening)"
    equalized_gray_reconstructed = deepcopy(equalized_gray)
    equalized_gray_reconstructed[landmask.dilated] .= 0
    equalized_gray_reconstructed = reconstruct(
        equalized_gray_reconstructed, structuring_elements.se_disk1, "dilation", true
    )
    equalized_gray_reconstructed[landmask.dilated] .= 0
end

imshow(equalized_gray_reconstructed/255)

In [ ]:
begin
    @debug "STEP 7: Brighten equalized_gray"
    brighten = get_brighten_mask(equalized_gray_reconstructed, gammagreen)
    equalized_gray[landmask.dilated] .= 0
    equalized_gray .= imbrighten(equalized_gray, brighten, brighten_factor)
end

imshow(equalized_gray/255)


In [ ]:
begin
    @debug "STEP 8: Get morphed_residue and adjust its gamma"
    morphed_residue = clamp.(equalized_gray - equalized_gray_reconstructed, 0, 255)

    agp = adjust_gamma_params
    equalized_gray_sharpened_reconstructed_adjusted = imcomplement(
        adjustgamma(equalized_gray_sharpened_reconstructed, agp.gamma)
    )
    adjusting_mask =
        equalized_gray_sharpened_reconstructed_adjusted .> agp.gamma_threshold
    morphed_residue[adjusting_mask] .=
        to_uint8.(morphed_residue[adjusting_mask] .* agp.gamma_factor)
end

imshow(morphed_residue/255)



In [ ]:
begin
    @debug "Step 9: Get preliminary ice masks"
    prelim_icemask, binarized_tiling = get_ice_masks(
        ref_image, morphed_residue, landmask.dilated, tiles, true; ice_masks_params...
    )
end
nothing # suppress output


In [ ]:
# View prelim_icemask
imshow(prelim_icemask)

In [ ]:
# View binarized_tiling
imshow(binarized_tiling)

In [ ]:

begin
    @debug "Step 10: Get segmentation mask from preliminary icemask"
    # Fill holes function in get_segment_mask a bit more aggressive than Matlabs
    segment_mask = get_segment_mask(prelim_icemask, binarized_tiling)
end

imshow(segment_mask)


In [ ]:
begin
    @debug "Step 11: Get local_maxima_mask and L0mask via watershed"
    local_maxima_mask, L0mask = watershed2(
        morphed_residue, segment_mask, prelim_icemask
    )
end
nothing # suppress output

In [ ]:
imshow(local_maxima_mask)

In [ ]:
imshow(L0mask)

In [ ]:

begin
    @debug "Step 12: Build icemask from all others"
    local_maxima_mask = to_uint8(local_maxima_mask * 255)
    prelim_icemask2 = _regularize(
        morphed_residue,
        local_maxima_mask,
        segment_mask,
        L0mask,
        structuring_elements.se_disk1;
        prelim_icemask_params...,
    )
end

imshow(prelim_icemask2/255)

In [ ]:
begin
    @debug "Step 13: Get improved icemask"
    icemask, _ = get_ice_masks(
        ref_image, prelim_icemask2, landmask.dilated, tiles, false; ice_masks_params...
    )
end

imshow(icemask)

In [ ]:
begin
    @debug "Step 14: Get final mask"
    se = structuring_elements
    se_erosion = se.se_disk1
    se_dilation = se.se_disk2
    final = get_final(icemask, segment_mask, se_erosion, se_dilation)
end

imshow(final)